In [13]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

#import pandas 
import pandas as pd

In [2]:
#set your executable path and url 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_mac64.zip
Driver has been saved in cache [/Users/kellykindla/.wdm/drivers/chromedriver/mac64/96.0.4664.45]


In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
#searching for elements with tag div and attribute list_text
# telling our browser to wait one second before searching for components 
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
#set up html parser 
html = browser.html
news_soup = soup(html, 'html.parser')
#create slide_elem to search for the <div /> tag 
slide_elem = news_soup.select_one('div.list_text')

In [5]:
#begin scraping by chaining find onto slide_elem to find specific data (class = "content_title")
slide_elem.find('div', class_='content_title')


<div class="content_title">NASA's Perseverance Rover Attached to Atlas V Rocket</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title` to get the title alone 
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Attached to Atlas V Rocket"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Ready for its launch later in the month, the Mars-bound rover will touch terra firma no more.'

### Featured Images

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button - clicks the second button tag
full_image_elem = browser.find_by_tag('button')[1]
#splinter will click the image
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url
#get('src') pulls the link to the most recent image 
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts Table 

In [14]:
#set up code to retrieve table of mars facts 
#create a dataframe from the first html table- index 0
df = pd.read_html('https://galaxyfacts-mars.com')[0]
#create/specify columns 
df.columns=['description', 'Mars', 'Earth']
#set the description column as the index 
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
#convert dataframe back to html ready code 
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
#end session 
browser.quit()